# Day 6: Distributed Systems Basics

## Week 22: System Design for Quantitative Finance

---

### Learning Objectives
- Understand CAP theorem and its implications for trading systems
- Implement hash-based and consistent hashing for data partitioning
- Build leader election algorithms for distributed coordination
- Create message passing systems between distributed nodes
- Implement CRDTs (Conflict-free Replicated Data Types)
- Master load balancing strategies for high availability
- Implement fault-tolerant retry logic with exponential backoff

### Why Distributed Systems Matter in Quant Finance
- **Scalability**: Process massive amounts of market data across multiple nodes
- **Low Latency**: Distribute computation closer to data sources
- **Fault Tolerance**: Ensure trading systems remain operational during failures
- **High Availability**: 24/7 market coverage requires redundant systems
- **Data Distribution**: Efficiently partition and replicate financial data

---

## 1. Import Required Libraries

In [1]:
"""
Distributed Systems Basics - Core Libraries
"""
import hashlib
import random
import time
import threading
import queue
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple, Any, Callable
from abc import ABC, abstractmethod
import bisect
from enum import Enum
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import heapq

# For visualization
import matplotlib.pyplot as plt
import numpy as np

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

print("✅ All libraries imported successfully!")
print("\nKey components for distributed systems:")
print("- hashlib: For consistent hashing")
print("- threading: For concurrent node simulation")
print("- queue: For message passing between nodes")
print("- collections: For distributed data structures")

✅ All libraries imported successfully!

Key components for distributed systems:
- hashlib: For consistent hashing
- threading: For concurrent node simulation
- queue: For message passing between nodes
- collections: For distributed data structures


## 2. Understanding CAP Theorem

The **CAP Theorem** states that a distributed system can only guarantee two of three properties:

- **Consistency (C)**: Every read receives the most recent write
- **Availability (A)**: Every request receives a response (success or failure)
- **Partition Tolerance (P)**: System continues to operate despite network partitions

### Trade-offs in Trading Systems:
| System Type | Priority | Example |
|------------|----------|---------|
| **CP** | Consistency + Partition Tolerance | Order matching engines, position tracking |
| **AP** | Availability + Partition Tolerance | Market data distribution, analytics |
| **CA** | Consistency + Availability | Single-node databases (no partition tolerance) |

In practice, network partitions are inevitable, so we choose between **CP** and **AP**.

In [2]:
"""
CAP Theorem Simulation - Distributed Key-Value Store
Demonstrates trade-offs between Consistency and Availability during partitions
"""

class CAPMode(Enum):
    CP = "Consistency + Partition Tolerance"
    AP = "Availability + Partition Tolerance"


@dataclass
class Node:
    """Represents a node in distributed system"""
    node_id: str
    data: Dict[str, Tuple[Any, int]] = field(default_factory=dict)  # key -> (value, version)
    is_partitioned: bool = False
    
    def write(self, key: str, value: Any, version: int) -> bool:
        """Write data to node"""
        if key not in self.data or self.data[key][1] < version:
            self.data[key] = (value, version)
            return True
        return False
    
    def read(self, key: str) -> Optional[Tuple[Any, int]]:
        """Read data from node"""
        return self.data.get(key)


class DistributedKVStore:
    """
    Distributed Key-Value Store demonstrating CAP theorem trade-offs
    """
    def __init__(self, num_nodes: int = 3, mode: CAPMode = CAPMode.CP):
        self.nodes = {f"node_{i}": Node(f"node_{i}") for i in range(num_nodes)}
        self.mode = mode
        self.version_counter = 0
        self.leader = "node_0"  # Simple leader election
        
    def write(self, key: str, value: Any) -> Tuple[bool, str]:
        """
        Write to distributed store
        - CP mode: Requires majority consensus, fails during partition
        - AP mode: Writes to available nodes, may have inconsistency
        """
        self.version_counter += 1
        version = self.version_counter
        
        available_nodes = [n for n in self.nodes.values() if not n.is_partitioned]
        total_nodes = len(self.nodes)
        majority = total_nodes // 2 + 1
        
        if self.mode == CAPMode.CP:
            # CP: Require majority consensus
            if len(available_nodes) < majority:
                return False, f"UNAVAILABLE: Only {len(available_nodes)}/{total_nodes} nodes reachable (need {majority})"
            
            # Write to majority
            success_count = 0
            for node in available_nodes[:majority]:
                if node.write(key, value, version):
                    success_count += 1
            
            if success_count >= majority:
                # Replicate to remaining available nodes asynchronously
                for node in available_nodes[majority:]:
                    node.write(key, value, version)
                return True, f"SUCCESS: Written to {len(available_nodes)} nodes with version {version}"
            return False, "FAILED: Could not achieve consensus"
            
        else:  # AP mode
            # AP: Write to any available node, accept potential inconsistency
            if not available_nodes:
                return False, "UNAVAILABLE: No nodes reachable"
            
            for node in available_nodes:
                node.write(key, value, version)
            
            return True, f"SUCCESS: Written to {len(available_nodes)} available nodes (may be inconsistent)"
    
    def read(self, key: str) -> Tuple[Optional[Any], str]:
        """
        Read from distributed store
        - CP mode: Read from leader/majority, fail if unavailable
        - AP mode: Read from any available node
        """
        available_nodes = [n for n in self.nodes.values() if not n.is_partitioned]
        
        if self.mode == CAPMode.CP:
            # CP: Require quorum read
            majority = len(self.nodes) // 2 + 1
            if len(available_nodes) < majority:
                return None, f"UNAVAILABLE: Cannot guarantee consistency"
            
            # Read from multiple nodes and return highest version
            values = []
            for node in available_nodes:
                result = node.read(key)
                if result:
                    values.append(result)
            
            if values:
                # Return value with highest version (most recent)
                latest = max(values, key=lambda x: x[1])
                return latest[0], f"CONSISTENT READ: value={latest[0]}, version={latest[1]}"
            return None, "KEY NOT FOUND"
            
        else:  # AP mode
            # AP: Return first available value (may be stale)
            for node in available_nodes:
                result = node.read(key)
                if result:
                    return result[0], f"AVAILABLE READ: value={result[0]} (may be stale)"
            return None, "KEY NOT FOUND or NO NODES AVAILABLE"
    
    def create_partition(self, partitioned_nodes: List[str]):
        """Simulate network partition"""
        for node_id in partitioned_nodes:
            if node_id in self.nodes:
                self.nodes[node_id].is_partitioned = True
                
    def heal_partition(self):
        """Heal network partition"""
        for node in self.nodes.values():
            node.is_partitioned = False
    
    def get_status(self) -> str:
        """Get cluster status"""
        available = sum(1 for n in self.nodes.values() if not n.is_partitioned)
        return f"Mode: {self.mode.value} | Available: {available}/{len(self.nodes)} nodes"


# Demonstrate CAP trade-offs
print("=" * 70)
print("CAP THEOREM DEMONSTRATION")
print("=" * 70)

# CP System Demo
print("\n📊 CP SYSTEM (Consistency + Partition Tolerance)")
print("-" * 50)
cp_store = DistributedKVStore(num_nodes=3, mode=CAPMode.CP)

# Normal operation
success, msg = cp_store.write("AAPL_price", 150.25)
print(f"Write AAPL_price=150.25: {msg}")

value, msg = cp_store.read("AAPL_price")
print(f"Read AAPL_price: {msg}")

# Create partition (minority)
print("\n🔴 Creating network partition (1 node isolated)...")
cp_store.create_partition(["node_2"])
print(f"Status: {cp_store.get_status()}")

success, msg = cp_store.write("AAPL_price", 151.00)
print(f"Write during partition: {msg}")

# Create partition (majority)
print("\n🔴 Extending partition (2 nodes isolated)...")
cp_store.create_partition(["node_1", "node_2"])
print(f"Status: {cp_store.get_status()}")

success, msg = cp_store.write("AAPL_price", 152.00)
print(f"Write during majority partition: {msg}")

value, msg = cp_store.read("AAPL_price")
print(f"Read during majority partition: {msg}")

# Heal and show consistency
print("\n🟢 Healing partition...")
cp_store.heal_partition()
print(f"Status: {cp_store.get_status()}")

# AP System Demo
print("\n" + "=" * 70)
print("📊 AP SYSTEM (Availability + Partition Tolerance)")
print("-" * 50)
ap_store = DistributedKVStore(num_nodes=3, mode=CAPMode.AP)

# Normal operation
success, msg = ap_store.write("GOOGL_price", 2800.50)
print(f"Write GOOGL_price=2800.50: {msg}")

# Create partition
print("\n🔴 Creating network partition (2 nodes isolated)...")
ap_store.create_partition(["node_1", "node_2"])
print(f"Status: {ap_store.get_status()}")

success, msg = ap_store.write("GOOGL_price", 2850.00)
print(f"Write during partition: {msg}")
print("⚠️  Note: Only node_0 has new value, others have stale data!")

value, msg = ap_store.read("GOOGL_price")
print(f"Read during partition: {msg}")

CAP THEOREM DEMONSTRATION

📊 CP SYSTEM (Consistency + Partition Tolerance)
--------------------------------------------------
Write AAPL_price=150.25: SUCCESS: Written to 3 nodes with version 1
Read AAPL_price: CONSISTENT READ: value=150.25, version=1

🔴 Creating network partition (1 node isolated)...
Status: Mode: Consistency + Partition Tolerance | Available: 2/3 nodes
Write during partition: SUCCESS: Written to 2 nodes with version 2

🔴 Extending partition (2 nodes isolated)...
Status: Mode: Consistency + Partition Tolerance | Available: 1/3 nodes
Write during majority partition: UNAVAILABLE: Only 1/3 nodes reachable (need 2)
Read during majority partition: UNAVAILABLE: Cannot guarantee consistency

🟢 Healing partition...
Status: Mode: Consistency + Partition Tolerance | Available: 3/3 nodes

📊 AP SYSTEM (Availability + Partition Tolerance)
--------------------------------------------------
Write GOOGL_price=2800.50: SUCCESS: Written to 3 available nodes (may be inconsistent)

🔴 Cre

## 3. Hash-Based Partitioning (Sharding)

**Data Partitioning** distributes data across multiple nodes to achieve:
- **Horizontal Scalability**: Handle more data by adding nodes
- **Parallel Processing**: Distribute computation across nodes
- **Load Distribution**: Balance workload evenly

### Simple Hash Partitioning
```
partition = hash(key) % num_partitions
```

**Use Case in Trading**: Partition order books by symbol hash for parallel processing.

In [3]:
"""
Simple Hash-Based Partitioning for Financial Data
"""

class HashPartitioner:
    """
    Simple modulo-based hash partitioner
    Distributes data across N partitions using hash(key) % N
    """
    def __init__(self, num_partitions: int):
        self.num_partitions = num_partitions
        self.partitions: Dict[int, List[Tuple[str, Any]]] = {
            i: [] for i in range(num_partitions)
        }
        
    def _hash(self, key: str) -> int:
        """Generate hash value for key"""
        return int(hashlib.md5(key.encode()).hexdigest(), 16)
    
    def get_partition(self, key: str) -> int:
        """Determine which partition a key belongs to"""
        return self._hash(key) % self.num_partitions
    
    def put(self, key: str, value: Any) -> int:
        """Store key-value pair in appropriate partition"""
        partition_id = self.get_partition(key)
        self.partitions[partition_id].append((key, value))
        return partition_id
    
    def get_partition_data(self, partition_id: int) -> List[Tuple[str, Any]]:
        """Get all data from a partition"""
        return self.partitions.get(partition_id, [])
    
    def get_distribution(self) -> Dict[int, int]:
        """Get count of items per partition"""
        return {pid: len(data) for pid, data in self.partitions.items()}


# Example: Partition trading symbols across nodes
print("=" * 70)
print("HASH-BASED PARTITIONING - TRADING SYMBOLS")
print("=" * 70)

# Create partitioner with 4 partitions (simulating 4 nodes)
partitioner = HashPartitioner(num_partitions=4)

# Sample trading symbols
symbols = [
    "AAPL", "GOOGL", "MSFT", "AMZN", "META", "NVDA", "TSLA", "JPM",
    "V", "JNJ", "WMT", "PG", "MA", "UNH", "HD", "DIS", "BAC", "XOM",
    "CSCO", "ADBE", "CRM", "NFLX", "INTC", "PFE", "KO", "PEP", "MRK",
    "ABBV", "TMO", "ACN", "AVGO", "COST", "MCD", "NKE", "LLY", "AMD"
]

# Distribute symbols
print("\nDistributing symbols across 4 partitions:")
print("-" * 50)

symbol_partitions = {}
for symbol in symbols:
    partition = partitioner.put(symbol, {"price": random.uniform(50, 500)})
    symbol_partitions[symbol] = partition

# Show distribution
print("\nPartition assignments:")
for pid in range(4):
    partition_symbols = [s for s, p in symbol_partitions.items() if p == pid]
    print(f"  Partition {pid}: {partition_symbols}")

# Visualize distribution
distribution = partitioner.get_distribution()
print(f"\n📊 Distribution: {distribution}")

# Check balance
counts = list(distribution.values())
print(f"   Min: {min(counts)}, Max: {max(counts)}, Std: {np.std(counts):.2f}")

# Problem: What happens when we add/remove a partition?
print("\n" + "=" * 70)
print("⚠️  PROBLEM: Rehashing when partitions change")
print("=" * 70)

# Original assignments
print("\nOriginal (4 partitions):")
for symbol in symbols[:8]:
    print(f"  {symbol} -> Partition {partitioner.get_partition(symbol)}")

# New partitioner with 5 partitions
new_partitioner = HashPartitioner(num_partitions=5)
print("\nAfter adding 1 partition (5 total):")
changes = 0
for symbol in symbols[:8]:
    old_part = partitioner.get_partition(symbol)
    new_part = new_partitioner.get_partition(symbol)
    changed = "🔴 MOVED!" if old_part != new_part else ""
    if changed:
        changes += 1
    print(f"  {symbol}: {old_part} -> {new_part} {changed}")

print(f"\n⚠️  {changes}/8 symbols ({changes/8*100:.0f}%) need to be relocated!")
print("This is expensive! Consistent Hashing solves this problem.")

HASH-BASED PARTITIONING - TRADING SYMBOLS

Distributing symbols across 4 partitions:
--------------------------------------------------

Partition assignments:
  Partition 0: ['NVDA', 'WMT', 'MA', 'BAC', 'INTC', 'PFE', 'ABBV', 'AVGO', 'COST']
  Partition 1: ['AMZN', 'PG', 'UNH', 'CSCO', 'PEP', 'TMO']
  Partition 2: ['AAPL', 'MSFT', 'V', 'ADBE', 'KO', 'AMD']
  Partition 3: ['GOOGL', 'META', 'TSLA', 'JPM', 'JNJ', 'HD', 'DIS', 'XOM', 'CRM', 'NFLX', 'MRK', 'ACN', 'MCD', 'NKE', 'LLY']

📊 Distribution: {0: 9, 1: 6, 2: 6, 3: 15}
   Min: 6, Max: 15, Std: 3.67

⚠️  PROBLEM: Rehashing when partitions change

Original (4 partitions):
  AAPL -> Partition 2
  GOOGL -> Partition 3
  MSFT -> Partition 2
  AMZN -> Partition 1
  META -> Partition 3
  NVDA -> Partition 0
  TSLA -> Partition 3
  JPM -> Partition 3

After adding 1 partition (5 total):
  AAPL: 2 -> 0 🔴 MOVED!
  GOOGL: 3 -> 1 🔴 MOVED!
  MSFT: 2 -> 0 🔴 MOVED!
  AMZN: 1 -> 3 🔴 MOVED!
  META: 3 -> 4 🔴 MOVED!
  NVDA: 0 -> 4 🔴 MOVED!
  TSLA: 3 -

## 4. Consistent Hashing

**Consistent Hashing** minimizes data movement when nodes are added or removed.

### How it works:
1. Both nodes and keys are hashed onto a circular ring (0 to 2^n)
2. Each key is assigned to the first node clockwise from its position
3. When a node is added/removed, only keys adjacent to it need to move

### Benefits:
- **Minimal Redistribution**: Only K/N keys move (K=keys, N=nodes)
- **Virtual Nodes**: Improve balance by adding multiple points per node
- **Hot Spot Prevention**: Even distribution across physical nodes

**Trading Use Case**: Distribute market data feeds across cache servers.

In [4]:
"""
Consistent Hashing Implementation with Virtual Nodes
"""

class ConsistentHashRing:
    """
    Consistent Hashing Ring Implementation
    
    Features:
    - Virtual nodes for better load distribution
    - O(log N) lookup using binary search
    - Minimal key redistribution on node changes
    """
    
    def __init__(self, virtual_nodes: int = 100):
        """
        Args:
            virtual_nodes: Number of virtual nodes per physical node
        """
        self.virtual_nodes = virtual_nodes
        self.ring: Dict[int, str] = {}  # hash position -> node_id
        self.sorted_keys: List[int] = []  # sorted hash positions
        self.nodes: set = set()
        
    def _hash(self, key: str) -> int:
        """Generate hash value in range [0, 2^32)"""
        return int(hashlib.sha256(key.encode()).hexdigest()[:8], 16)
    
    def add_node(self, node_id: str) -> List[int]:
        """
        Add a node to the ring with virtual nodes
        Returns: List of hash positions added
        """
        if node_id in self.nodes:
            return []
            
        self.nodes.add(node_id)
        positions = []
        
        for i in range(self.virtual_nodes):
            # Create unique virtual node identifier
            virtual_key = f"{node_id}#vn{i}"
            hash_val = self._hash(virtual_key)
            
            self.ring[hash_val] = node_id
            positions.append(hash_val)
            
            # Maintain sorted order for binary search
            bisect.insort(self.sorted_keys, hash_val)
        
        return positions
    
    def remove_node(self, node_id: str) -> List[int]:
        """
        Remove a node from the ring
        Returns: List of hash positions removed
        """
        if node_id not in self.nodes:
            return []
            
        self.nodes.remove(node_id)
        positions_removed = []
        
        for i in range(self.virtual_nodes):
            virtual_key = f"{node_id}#vn{i}"
            hash_val = self._hash(virtual_key)
            
            if hash_val in self.ring:
                del self.ring[hash_val]
                self.sorted_keys.remove(hash_val)
                positions_removed.append(hash_val)
        
        return positions_removed
    
    def get_node(self, key: str) -> Optional[str]:
        """
        Get the node responsible for a key
        Uses binary search to find first node clockwise from key's hash
        """
        if not self.ring:
            return None
            
        hash_val = self._hash(key)
        
        # Binary search for first position >= hash_val
        idx = bisect.bisect_left(self.sorted_keys, hash_val)
        
        # Wrap around to first node if past the end
        if idx >= len(self.sorted_keys):
            idx = 0
            
        return self.ring[self.sorted_keys[idx]]
    
    def get_nodes_for_key(self, key: str, num_replicas: int = 3) -> List[str]:
        """
        Get multiple nodes for replication
        Returns N distinct physical nodes for a key
        """
        if not self.ring or num_replicas > len(self.nodes):
            return list(self.nodes)[:num_replicas]
            
        hash_val = self._hash(key)
        idx = bisect.bisect_left(self.sorted_keys, hash_val)
        
        nodes = []
        seen = set()
        
        for i in range(len(self.sorted_keys)):
            pos = (idx + i) % len(self.sorted_keys)
            node = self.ring[self.sorted_keys[pos]]
            
            if node not in seen:
                nodes.append(node)
                seen.add(node)
                
            if len(nodes) >= num_replicas:
                break
                
        return nodes


# Demonstrate Consistent Hashing
print("=" * 70)
print("CONSISTENT HASHING DEMONSTRATION")
print("=" * 70)

# Create ring with 50 virtual nodes per physical node
ring = ConsistentHashRing(virtual_nodes=50)

# Add initial nodes
initial_nodes = ["cache_server_1", "cache_server_2", "cache_server_3"]
for node in initial_nodes:
    ring.add_node(node)
    
print(f"\n📊 Ring created with {len(initial_nodes)} nodes ({ring.virtual_nodes} virtual nodes each)")
print(f"   Total ring positions: {len(ring.sorted_keys)}")

# Assign symbols to nodes
symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "META", "NVDA", "TSLA", "JPM",
           "V", "JNJ", "WMT", "PG", "MA", "UNH", "HD", "DIS"]

print("\n🔗 Symbol -> Node Assignments:")
original_assignments = {}
for symbol in symbols:
    node = ring.get_node(symbol)
    original_assignments[symbol] = node

# Count per node
node_counts = defaultdict(int)
for node in original_assignments.values():
    node_counts[node] += 1
    
print(f"   Distribution: {dict(node_counts)}")

# Show replication
print("\n📋 Replication (3 replicas per key):")
for symbol in symbols[:4]:
    replicas = ring.get_nodes_for_key(symbol, num_replicas=3)
    print(f"   {symbol}: {replicas}")

# Add a new node
print("\n" + "=" * 70)
print("📥 ADDING NEW NODE: cache_server_4")
print("=" * 70)

ring.add_node("cache_server_4")

# Check reassignments
moved = 0
print("\nKey reassignments:")
for symbol in symbols:
    old_node = original_assignments[symbol]
    new_node = ring.get_node(symbol)
    if old_node != new_node:
        moved += 1
        print(f"   🔄 {symbol}: {old_node} -> {new_node}")

print(f"\n✅ Only {moved}/{len(symbols)} keys ({moved/len(symbols)*100:.1f}%) needed to move!")
print("   (With simple hash partitioning, ~75% would have moved)")

# Show new distribution
new_counts = defaultdict(int)
for symbol in symbols:
    new_counts[ring.get_node(symbol)] += 1
print(f"   New distribution: {dict(new_counts)}")

# Remove a node
print("\n" + "=" * 70)
print("📤 REMOVING NODE: cache_server_2")
print("=" * 70)

# Get current assignments before removal
pre_removal = {s: ring.get_node(s) for s in symbols}
ring.remove_node("cache_server_2")

# Check reassignments
moved = 0
print("\nKey reassignments from removed node:")
for symbol in symbols:
    old_node = pre_removal[symbol]
    new_node = ring.get_node(symbol)
    if old_node != new_node:
        moved += 1
        print(f"   🔄 {symbol}: {old_node} -> {new_node}")

print(f"\n✅ {moved} keys from the removed node redistributed to remaining nodes")

CONSISTENT HASHING DEMONSTRATION

📊 Ring created with 3 nodes (50 virtual nodes each)
   Total ring positions: 150

🔗 Symbol -> Node Assignments:
   Distribution: {'cache_server_1': 11, 'cache_server_2': 3, 'cache_server_3': 2}

📋 Replication (3 replicas per key):
   AAPL: ['cache_server_1', 'cache_server_3', 'cache_server_2']
   GOOGL: ['cache_server_2', 'cache_server_3', 'cache_server_1']
   MSFT: ['cache_server_1', 'cache_server_3', 'cache_server_2']
   AMZN: ['cache_server_1', 'cache_server_2', 'cache_server_3']

📥 ADDING NEW NODE: cache_server_4

Key reassignments:
   🔄 AAPL: cache_server_1 -> cache_server_4
   🔄 GOOGL: cache_server_2 -> cache_server_4
   🔄 AMZN: cache_server_1 -> cache_server_4
   🔄 TSLA: cache_server_1 -> cache_server_4
   🔄 MA: cache_server_3 -> cache_server_4
   🔄 UNH: cache_server_2 -> cache_server_4
   🔄 HD: cache_server_1 -> cache_server_4

✅ Only 7/16 keys (43.8%) needed to move!
   (With simple hash partitioning, ~75% would have moved)
   New distribution

## 5. Leader Election - Bully Algorithm

**Leader Election** determines which node coordinates distributed operations.

### Bully Algorithm:
1. When a node detects the leader has failed, it starts an election
2. It sends ELECTION messages to all nodes with higher IDs
3. If no higher node responds, it becomes the leader
4. The node with the highest ID that responds wins

### Trading System Use Cases:
- **Order Matching**: Single leader ensures consistent order matching
- **Position Aggregation**: Leader coordinates position calculations
- **Failover**: Automatic leader election when primary fails

In [5]:
"""
Bully Algorithm for Leader Election
"""

class BullyNode:
    """
    Node implementing the Bully Algorithm for leader election
    """
    def __init__(self, node_id: int, cluster: 'BullyCluster'):
        self.node_id = node_id
        self.cluster = cluster
        self.is_alive = True
        self.current_leader: Optional[int] = None
        self.election_in_progress = False
        
    def start_election(self) -> int:
        """
        Start an election process
        Returns the ID of the elected leader
        """
        print(f"  📢 Node {self.node_id}: Starting election...")
        self.election_in_progress = True
        
        # Find all nodes with higher IDs
        higher_nodes = [
            node for node in self.cluster.get_alive_nodes()
            if node.node_id > self.node_id
        ]
        
        if not higher_nodes:
            # No higher nodes, I am the leader!
            print(f"  👑 Node {self.node_id}: No higher nodes responding. I am the leader!")
            self.become_leader()
            return self.node_id
        
        # Send election message to higher nodes
        print(f"  📤 Node {self.node_id}: Sending ELECTION to nodes {[n.node_id for n in higher_nodes]}")
        
        responses = []
        for node in higher_nodes:
            response = node.receive_election(self.node_id)
            if response:
                responses.append(node.node_id)
        
        if not responses:
            # No responses, I am the leader
            print(f"  👑 Node {self.node_id}: No responses received. I am the leader!")
            self.become_leader()
            return self.node_id
        else:
            # Higher node(s) responded, wait for leader announcement
            print(f"  📥 Node {self.node_id}: Received OK from nodes {responses}. Waiting...")
            # The highest responding node should become leader
            return max(responses)
    
    def receive_election(self, from_node: int) -> bool:
        """
        Receive election message from lower node
        Returns True (OK) if alive
        """
        if not self.is_alive:
            return False
            
        print(f"  📥 Node {self.node_id}: Received ELECTION from Node {from_node}. Sending OK.")
        
        # Start my own election since I have higher ID
        if not self.election_in_progress:
            self.start_election()
            
        return True
    
    def become_leader(self):
        """Announce leadership to all nodes"""
        self.current_leader = self.node_id
        self.election_in_progress = False
        
        # Broadcast to all nodes
        for node in self.cluster.get_alive_nodes():
            node.receive_leader_announcement(self.node_id)
    
    def receive_leader_announcement(self, leader_id: int):
        """Receive leader announcement"""
        self.current_leader = leader_id
        self.election_in_progress = False
        if self.node_id != leader_id:
            print(f"  📢 Node {self.node_id}: Acknowledged Node {leader_id} as leader")
    
    def fail(self):
        """Simulate node failure"""
        self.is_alive = False
        print(f"  💀 Node {self.node_id}: FAILED!")
    
    def recover(self):
        """Recover a failed node"""
        self.is_alive = True
        print(f"  🔄 Node {self.node_id}: RECOVERED!")


class BullyCluster:
    """Cluster of nodes using Bully Algorithm"""
    
    def __init__(self, num_nodes: int):
        self.nodes = {}
        for i in range(num_nodes):
            self.nodes[i] = BullyNode(i, self)
            
    def get_alive_nodes(self) -> List[BullyNode]:
        """Get list of alive nodes"""
        return [n for n in self.nodes.values() if n.is_alive]
    
    def get_leader(self) -> Optional[int]:
        """Get current leader ID"""
        alive = self.get_alive_nodes()
        if alive:
            return alive[0].current_leader
        return None
    
    def initial_election(self):
        """Run initial election from lowest node"""
        lowest_alive = min(self.get_alive_nodes(), key=lambda n: n.node_id)
        lowest_alive.start_election()


# Demonstrate Bully Algorithm
print("=" * 70)
print("BULLY ALGORITHM - LEADER ELECTION")
print("=" * 70)

# Create cluster with 5 nodes (IDs: 0, 1, 2, 3, 4)
cluster = BullyCluster(num_nodes=5)

print("\n🏁 INITIAL ELECTION")
print("-" * 50)
cluster.initial_election()
print(f"\n✅ Leader: Node {cluster.get_leader()}")

# Simulate leader failure
print("\n" + "=" * 70)
print("💀 LEADER FAILURE SCENARIO")
print("=" * 70)

print("\nNode 4 (leader) fails...")
cluster.nodes[4].fail()

print("\nNode 2 detects failure and starts election:")
print("-" * 50)
cluster.nodes[2].start_election()
print(f"\n✅ New Leader: Node {cluster.get_leader()}")

# Another failure
print("\n" + "=" * 70)
print("💀 CASCADING FAILURE")
print("=" * 70)

print("\nNode 3 (current leader) also fails...")
cluster.nodes[3].fail()

print("\nNode 1 detects failure and starts election:")
print("-" * 50)
cluster.nodes[1].start_election()
print(f"\n✅ New Leader: Node {cluster.get_leader()}")

# Node recovery
print("\n" + "=" * 70)
print("🔄 NODE RECOVERY")
print("=" * 70)

print("\nNode 4 recovers and starts election:")
print("-" * 50)
cluster.nodes[4].recover()
cluster.nodes[4].start_election()
print(f"\n✅ Leader after recovery: Node {cluster.get_leader()}")

BULLY ALGORITHM - LEADER ELECTION

🏁 INITIAL ELECTION
--------------------------------------------------
  📢 Node 0: Starting election...
  📤 Node 0: Sending ELECTION to nodes [1, 2, 3, 4]
  📥 Node 1: Received ELECTION from Node 0. Sending OK.
  📢 Node 1: Starting election...
  📤 Node 1: Sending ELECTION to nodes [2, 3, 4]
  📥 Node 2: Received ELECTION from Node 1. Sending OK.
  📢 Node 2: Starting election...
  📤 Node 2: Sending ELECTION to nodes [3, 4]
  📥 Node 3: Received ELECTION from Node 2. Sending OK.
  📢 Node 3: Starting election...
  📤 Node 3: Sending ELECTION to nodes [4]
  📥 Node 4: Received ELECTION from Node 3. Sending OK.
  📢 Node 4: Starting election...
  👑 Node 4: No higher nodes responding. I am the leader!
  📢 Node 0: Acknowledged Node 4 as leader
  📢 Node 1: Acknowledged Node 4 as leader
  📢 Node 2: Acknowledged Node 4 as leader
  📢 Node 3: Acknowledged Node 4 as leader
  📥 Node 3: Received OK from nodes [4]. Waiting...
  📥 Node 4: Received ELECTION from Node 2. Sendi

## 6. Message Passing Between Nodes

**Message Passing** is fundamental to distributed systems communication.

### Common Patterns:
- **Request-Response**: Synchronous communication
- **Publish-Subscribe**: Asynchronous, decoupled messaging
- **Fire-and-Forget**: Async without acknowledgment

### Trading System Applications:
- Market data distribution (pub-sub)
- Order routing (request-response)
- Risk alerts (fire-and-forget with retry)

In [6]:
"""
Message Passing System Simulation
Demonstrates distributed communication patterns
"""

@dataclass
class Message:
    """Message structure for inter-node communication"""
    msg_id: str
    msg_type: str  # 'market_data', 'order', 'ack', 'heartbeat'
    sender: str
    receiver: str
    payload: Dict[str, Any]
    timestamp: float = field(default_factory=time.time)
    
    def to_dict(self) -> dict:
        return {
            'msg_id': self.msg_id,
            'msg_type': self.msg_type,
            'sender': self.sender,
            'receiver': self.receiver,
            'payload': self.payload,
            'timestamp': self.timestamp
        }


class MessageBroker:
    """
    Simple message broker for pub-sub and point-to-point messaging
    """
    def __init__(self, failure_rate: float = 0.0, latency_ms: Tuple[int, int] = (1, 10)):
        """
        Args:
            failure_rate: Probability of message loss (0-1)
            latency_ms: Range of simulated network latency
        """
        self.queues: Dict[str, queue.Queue] = defaultdict(queue.Queue)
        self.topics: Dict[str, List[str]] = defaultdict(list)  # topic -> subscribers
        self.failure_rate = failure_rate
        self.latency_ms = latency_ms
        self.message_log: List[Dict] = []
        
    def subscribe(self, topic: str, subscriber_id: str):
        """Subscribe to a topic"""
        if subscriber_id not in self.topics[topic]:
            self.topics[topic].append(subscriber_id)
            print(f"  ✅ {subscriber_id} subscribed to '{topic}'")
    
    def unsubscribe(self, topic: str, subscriber_id: str):
        """Unsubscribe from a topic"""
        if subscriber_id in self.topics[topic]:
            self.topics[topic].remove(subscriber_id)
            
    def publish(self, topic: str, message: Message) -> int:
        """
        Publish message to all subscribers of a topic
        Returns number of successful deliveries
        """
        subscribers = self.topics.get(topic, [])
        delivered = 0
        
        for subscriber in subscribers:
            # Simulate network conditions
            if random.random() < self.failure_rate:
                print(f"  ❌ Message {message.msg_id} LOST to {subscriber}")
                continue
                
            # Simulate latency
            latency = random.randint(*self.latency_ms) / 1000
            time.sleep(latency)
            
            # Deliver message
            self.queues[subscriber].put(message)
            delivered += 1
            
        self.message_log.append({
            'action': 'publish',
            'topic': topic,
            'msg_id': message.msg_id,
            'delivered': delivered,
            'total_subscribers': len(subscribers)
        })
        
        return delivered
    
    def send_direct(self, message: Message) -> bool:
        """
        Send message directly to a specific receiver
        Returns True if delivered
        """
        if random.random() < self.failure_rate:
            print(f"  ❌ Message {message.msg_id} LOST")
            return False
            
        latency = random.randint(*self.latency_ms) / 1000
        time.sleep(latency)
        
        self.queues[message.receiver].put(message)
        self.message_log.append({
            'action': 'direct',
            'msg_id': message.msg_id,
            'from': message.sender,
            'to': message.receiver,
            'delivered': True
        })
        return True
    
    def receive(self, receiver_id: str, timeout: float = 1.0) -> Optional[Message]:
        """Receive a message (blocking with timeout)"""
        try:
            return self.queues[receiver_id].get(timeout=timeout)
        except queue.Empty:
            return None
    
    def receive_all(self, receiver_id: str) -> List[Message]:
        """Receive all pending messages"""
        messages = []
        while not self.queues[receiver_id].empty():
            messages.append(self.queues[receiver_id].get())
        return messages


class TradingNode:
    """Simulated trading system node"""
    
    def __init__(self, node_id: str, broker: MessageBroker):
        self.node_id = node_id
        self.broker = broker
        self.received_messages: List[Message] = []
        self.msg_counter = 0
        
    def create_message(self, msg_type: str, receiver: str, payload: dict) -> Message:
        """Create a new message"""
        self.msg_counter += 1
        return Message(
            msg_id=f"{self.node_id}-{self.msg_counter}",
            msg_type=msg_type,
            sender=self.node_id,
            receiver=receiver,
            payload=payload
        )
    
    def send_market_data(self, symbol: str, price: float, volume: int):
        """Publish market data update"""
        msg = self.create_message(
            msg_type='market_data',
            receiver='broadcast',
            payload={'symbol': symbol, 'price': price, 'volume': volume}
        )
        delivered = self.broker.publish(f'market_data.{symbol}', msg)
        print(f"  📤 {self.node_id}: Published {symbol} update to {delivered} subscribers")
        
    def send_order(self, order: dict, target_node: str):
        """Send order to specific node"""
        msg = self.create_message(
            msg_type='order',
            receiver=target_node,
            payload=order
        )
        success = self.broker.send_direct(msg)
        status = "✅ Delivered" if success else "❌ Failed"
        print(f"  📤 {self.node_id}: Sent order to {target_node} - {status}")
        
    def process_messages(self):
        """Process all pending messages"""
        messages = self.broker.receive_all(self.node_id)
        for msg in messages:
            self.received_messages.append(msg)
            print(f"  📥 {self.node_id}: Received {msg.msg_type} from {msg.sender}")
            

# Demonstrate Message Passing
print("=" * 70)
print("MESSAGE PASSING SYSTEM DEMONSTRATION")
print("=" * 70)

# Create broker with some simulated failures
broker = MessageBroker(failure_rate=0.1, latency_ms=(1, 5))

# Create nodes
market_data_feed = TradingNode("MarketDataFeed", broker)
strategy_1 = TradingNode("Strategy_Alpha", broker)
strategy_2 = TradingNode("Strategy_Beta", broker)
order_router = TradingNode("OrderRouter", broker)

# Setup subscriptions (pub-sub pattern)
print("\n📋 SETTING UP SUBSCRIPTIONS")
print("-" * 50)
broker.subscribe("market_data.AAPL", strategy_1.node_id)
broker.subscribe("market_data.AAPL", strategy_2.node_id)
broker.subscribe("market_data.GOOGL", strategy_1.node_id)

# Publish market data
print("\n📊 PUBLISHING MARKET DATA (Pub-Sub)")
print("-" * 50)
market_data_feed.send_market_data("AAPL", 150.25, 10000)
market_data_feed.send_market_data("GOOGL", 2800.50, 5000)

# Process received messages
print("\n📥 PROCESSING RECEIVED MESSAGES")
print("-" * 50)
strategy_1.process_messages()
strategy_2.process_messages()

# Direct messaging (point-to-point)
print("\n📤 DIRECT ORDER ROUTING (Point-to-Point)")
print("-" * 50)
strategy_1.send_order(
    {'symbol': 'AAPL', 'side': 'BUY', 'qty': 100, 'price': 150.00},
    order_router.node_id
)
strategy_2.send_order(
    {'symbol': 'GOOGL', 'side': 'SELL', 'qty': 50, 'price': 2805.00},
    order_router.node_id
)

# Order router processes orders
order_router.process_messages()

print(f"\n📊 Message Statistics:")
print(f"   Total messages logged: {len(broker.message_log)}")
print(f"   Strategy_Alpha received: {len(strategy_1.received_messages)}")
print(f"   Strategy_Beta received: {len(strategy_2.received_messages)}")
print(f"   OrderRouter received: {len(order_router.received_messages)}")

MESSAGE PASSING SYSTEM DEMONSTRATION

📋 SETTING UP SUBSCRIPTIONS
--------------------------------------------------
  ✅ Strategy_Alpha subscribed to 'market_data.AAPL'
  ✅ Strategy_Beta subscribed to 'market_data.AAPL'
  ✅ Strategy_Alpha subscribed to 'market_data.GOOGL'

📊 PUBLISHING MARKET DATA (Pub-Sub)
--------------------------------------------------
  📤 MarketDataFeed: Published AAPL update to 2 subscribers
  📤 MarketDataFeed: Published GOOGL update to 1 subscribers

📥 PROCESSING RECEIVED MESSAGES
--------------------------------------------------
  📥 Strategy_Alpha: Received market_data from MarketDataFeed
  📥 Strategy_Alpha: Received market_data from MarketDataFeed
  📥 Strategy_Beta: Received market_data from MarketDataFeed

📤 DIRECT ORDER ROUTING (Point-to-Point)
--------------------------------------------------
  ❌ Message Strategy_Alpha-1 LOST
  📤 Strategy_Alpha: Sent order to OrderRouter - ❌ Failed
  📤 Strategy_Beta: Sent order to OrderRouter - ✅ Delivered
  📥 OrderRouter

## 7. CRDTs - Conflict-free Replicated Data Types

**CRDTs** are data structures that can be replicated across multiple nodes and merged without conflicts.

### Key Property:
- All replicas converge to the same state regardless of message ordering
- No coordination needed (eventually consistent)

### Types:
- **G-Counter**: Grow-only counter (increment only)
- **PN-Counter**: Positive-Negative counter (increment and decrement)
- **G-Set**: Grow-only set (add only)
- **OR-Set**: Observed-Remove set (add and remove)

### Trading Use Case: 
Distributed trade counters, volume aggregation across data centers

In [7]:
"""
CRDT Implementations - Conflict-free Replicated Data Types
"""

class GCounter:
    """
    G-Counter (Grow-only Counter) CRDT
    
    Each node maintains its own counter. Total value is sum of all node counters.
    Merge operation takes maximum of each node's counter.
    """
    
    def __init__(self, node_id: str):
        self.node_id = node_id
        self.counts: Dict[str, int] = defaultdict(int)
        
    def increment(self, amount: int = 1):
        """Increment this node's counter"""
        self.counts[self.node_id] += amount
        
    def value(self) -> int:
        """Get total count across all nodes"""
        return sum(self.counts.values())
    
    def merge(self, other: 'GCounter'):
        """
        Merge with another G-Counter
        Takes maximum of each node's count
        """
        all_nodes = set(self.counts.keys()) | set(other.counts.keys())
        for node in all_nodes:
            self.counts[node] = max(self.counts[node], other.counts[node])
    
    def get_state(self) -> Dict[str, int]:
        """Get current state for serialization"""
        return dict(self.counts)
    
    def __repr__(self):
        return f"GCounter({self.node_id}, counts={dict(self.counts)}, total={self.value()})"


class PNCounter:
    """
    PN-Counter (Positive-Negative Counter) CRDT
    
    Uses two G-Counters: one for increments, one for decrements.
    Value = P_counter - N_counter
    """
    
    def __init__(self, node_id: str):
        self.node_id = node_id
        self.p_counter = GCounter(node_id)  # Positive increments
        self.n_counter = GCounter(node_id)  # Negative increments (decrements)
        
    def increment(self, amount: int = 1):
        """Increment the counter"""
        self.p_counter.increment(amount)
        
    def decrement(self, amount: int = 1):
        """Decrement the counter"""
        self.n_counter.increment(amount)
        
    def value(self) -> int:
        """Get net value"""
        return self.p_counter.value() - self.n_counter.value()
    
    def merge(self, other: 'PNCounter'):
        """Merge with another PN-Counter"""
        self.p_counter.merge(other.p_counter)
        self.n_counter.merge(other.n_counter)
        
    def __repr__(self):
        return f"PNCounter({self.node_id}, value={self.value()}, P={self.p_counter.value()}, N={self.n_counter.value()})"


class LWWRegister:
    """
    Last-Write-Wins Register CRDT
    
    Each write includes a timestamp. On merge, the value with the highest timestamp wins.
    Useful for single-value storage that can be updated.
    """
    
    def __init__(self, node_id: str):
        self.node_id = node_id
        self.value: Any = None
        self.timestamp: float = 0.0
        
    def set(self, value: Any, timestamp: Optional[float] = None):
        """Set value with timestamp"""
        ts = timestamp or time.time()
        if ts >= self.timestamp:
            self.value = value
            self.timestamp = ts
            
    def get(self) -> Any:
        """Get current value"""
        return self.value
    
    def merge(self, other: 'LWWRegister'):
        """Merge with another register - highest timestamp wins"""
        if other.timestamp > self.timestamp:
            self.value = other.value
            self.timestamp = other.timestamp
            
    def __repr__(self):
        return f"LWWRegister(value={self.value}, ts={self.timestamp:.3f})"


# Demonstrate CRDTs
print("=" * 70)
print("CRDT DEMONSTRATION - DISTRIBUTED TRADE COUNTERS")
print("=" * 70)

# Scenario: Counting trades across multiple data centers
print("\n📊 G-Counter: Distributed Trade Counter")
print("-" * 50)

# Create counters at different data centers
nyc_counter = GCounter("NYC")
london_counter = GCounter("London")
tokyo_counter = GCounter("Tokyo")

# Each data center processes trades independently
print("\nProcessing trades at each data center...")
nyc_counter.increment(150)      # 150 trades in NYC
london_counter.increment(120)   # 120 trades in London
tokyo_counter.increment(180)    # 180 trades in Tokyo

print(f"  NYC:    {nyc_counter}")
print(f"  London: {london_counter}")
print(f"  Tokyo:  {tokyo_counter}")

# Merge counters (simulating eventual consistency)
print("\nMerging counters (any order produces same result)...")

# NYC receives London's state
nyc_counter.merge(london_counter)
print(f"  NYC after merge with London: {nyc_counter}")

# NYC receives Tokyo's state
nyc_counter.merge(tokyo_counter)
print(f"  NYC after merge with Tokyo: {nyc_counter}")

# Tokyo also merges (different order)
tokyo_counter.merge(london_counter)
tokyo_counter.merge(nyc_counter)
print(f"  Tokyo after merges: {tokyo_counter}")

print(f"\n✅ Both converge to same total: {nyc_counter.value()} trades")

# PN-Counter Demo
print("\n" + "=" * 70)
print("📊 PN-Counter: Net Position Tracking")
print("-" * 50)

# Track net position (buys - sells) across nodes
node_a = PNCounter("Algo_A")
node_b = PNCounter("Algo_B")

# Node A: 100 buys, 30 sells
node_a.increment(100)
node_a.decrement(30)
print(f"Node A - Bought 100, Sold 30: {node_a}")

# Node B: 50 buys, 80 sells
node_b.increment(50)
node_b.decrement(80)
print(f"Node B - Bought 50, Sold 80: {node_b}")

# Merge
node_a.merge(node_b)
print(f"\nAfter merge: {node_a}")
print(f"Net position: {node_a.value()} (150 buys - 110 sells)")

# LWW Register Demo
print("\n" + "=" * 70)
print("📊 LWW-Register: Last Price Update Wins")
print("-" * 50)

# Track latest price across nodes
reg_1 = LWWRegister("Feed_1")
reg_2 = LWWRegister("Feed_2")

# Simulate price updates with timestamps
reg_1.set(150.25, timestamp=1.0)
print(f"Feed 1 sets price at t=1.0: {reg_1}")

reg_2.set(150.30, timestamp=2.0)
print(f"Feed 2 sets price at t=2.0: {reg_2}")

reg_1.set(150.20, timestamp=1.5)  # Earlier timestamp, should be ignored on merge
print(f"Feed 1 sets price at t=1.5: {reg_1}")

# Merge
print("\nMerging (latest timestamp wins)...")
reg_1.merge(reg_2)
print(f"Feed 1 after merge: {reg_1}")

reg_2.merge(reg_1)
print(f"Feed 2 after merge: {reg_2}")
print(f"\n✅ Both show price: ${reg_1.get()}")

CRDT DEMONSTRATION - DISTRIBUTED TRADE COUNTERS

📊 G-Counter: Distributed Trade Counter
--------------------------------------------------

Processing trades at each data center...
  NYC:    GCounter(NYC, counts={'NYC': 150}, total=150)
  London: GCounter(London, counts={'London': 120}, total=120)
  Tokyo:  GCounter(Tokyo, counts={'Tokyo': 180}, total=180)

Merging counters (any order produces same result)...
  NYC after merge with London: GCounter(NYC, counts={'NYC': 150, 'London': 120}, total=270)
  NYC after merge with Tokyo: GCounter(NYC, counts={'NYC': 150, 'London': 120, 'Tokyo': 180}, total=450)
  Tokyo after merges: GCounter(Tokyo, counts={'Tokyo': 180, 'London': 120, 'NYC': 150}, total=450)

✅ Both converge to same total: 450 trades

📊 PN-Counter: Net Position Tracking
--------------------------------------------------
Node A - Bought 100, Sold 30: PNCounter(Algo_A, value=70, P=100, N=30)
Node B - Bought 50, Sold 80: PNCounter(Algo_B, value=-30, P=50, N=80)

After merge: PNCou